In [13]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
from random import random
import numpy as np

In [14]:
df = pd.read_csv("SpotifyData.csv")

2012    115
2017    111
2001    108
2018    107
2010    107
2014    104
2005    104
2011     99
2016     99
2015     99
2003     97
2008     97
2004     96
2006     95
2007     94
2002     90
2019     89
2013     89
2009     84
2000     74
1999     38
2020      3
1998      1
Name: year, dtype: int64

In [19]:
#grouped.drop(['instrumentalness', 'acousticness', 'speechiness', 'key', 'mode', 'danceability', "artist", "song", "liveness"], axis = 1, inplace=True)
df.dropna(axis=0, inplace=True)
df['duration'] = df['duration'] / 1000
df

,artist,song,duration,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Britney Spears,Oops!...I Did It Again,211.160,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.3000,0.000018,0.3550,0.894,95.053
1,blink-182,All The Small Things,167.066,1999,79,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.000000,0.6120,0.684,148.726
2,Faith Hill,Breathe,250.546,1999,66,0.529,0.496,7,-9.007,1,0.0290,0.1730,0.000000,0.2510,0.278,136.859
3,Bon Jovi,It's My Life,224.493,2000,78,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.000013,0.3470,0.544,119.992
4,*NSYNC,Bye Bye Bye,200.560,2000,65,0.614,0.928,8,-4.806,0,0.0516,0.0408,0.001040,0.0845,0.879,172.656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Jonas Brothers,Sucker,181.026,2019,79,0.842,0.734,1,-5.065,0,0.0588,0.0427,0.000000,0.1060,0.952,137.958
1996,Taylor Swift,Cruel Summer,178.426,2019,78,0.552,0.702,9,-5.707,1,0.1570,0.1170,0.000021,0.1050,0.564,169.994
1997,Blanco Brown,The Git Up,200.593,2019,69,0.847,0.678,9,-8.635,1,0.1090,0.0669,0.000000,0.2740,0.811,97.984
1998,Sam Smith,Dancing With A Stranger (with Normani),171.029,2019,75,0.741,0.520,8,-7.513,1,0.0656,0.4500,0.000002,0.2220,0.347,102.998


In [21]:

y = df['popularity'].values
X = df[['duration', 'year', 'energy', 'loudness', 'valence', 'tempo']].values

In [35]:
length = len(df.index)
split = int(length * 0.7)
X_train = X[:split]
X_train = np.c_[np.ones(X_train.shape[0]), X_train]
X_test = X[split:]
X_test = np.c_[np.ones(X_test.shape[0]), X_test]
y_train = y[:split]
y_test = y[split:]
print(y_train)


[77 79 66 ... 67 83 57]
